<h1>Notebook to clean the Tigers Data Set</h1>
<p>data/DTIFanData_2-21-25.csv will be cleaned</p>

<h3>Dependencies</h3>

In [80]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
# import Helpers from Helpers.py which is in same directory
from Helpers import *

<h3>Load the dataset</h3>

In [60]:
path = 'data/DTIFanData_2-21-25.csv'
# Load the data
data = pd.read_csv(path)
# Display the first few rows of the dataframe
print(data.head())

/tmp/ipykernel_1457/3605361889.py:3: DtypeWarning: Columns (4,6,7,165) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(path)


   KeepFlag  GlobalKey  SeasonKey FanSinceDate  \
0         1         11       2025   2009-05-01   
1         1         12       2025   2009-02-25   
2         1         15       2025   2008-08-05   
3         1         17       2025   2007-06-24   
4         1         19       2025   2008-08-05   

                  FirstGameAttended  TotalGamesAttended  \
0         2023-08-05 Tampa Bay Rays                 4.0   
1  2024-06-25 Philadelphia Phillies                 1.0   
2     2013-06-27 Los Angeles Angels                 6.0   
3      2015-07-05 Toronto Blue Jays                 9.0   
4          2013-07-14 Texas Rangers                 1.0   

                    FirstGameBought                    LastGameBought  \
0         2023-08-05 Tampa Bay Rays  2024-06-26 Philadelphia Phillies   
1  2024-06-25 Philadelphia Phillies  2024-06-25 Philadelphia Phillies   
2       2021-06-08 Seattle Mariners       2021-06-08 Seattle Mariners   
3  2021-06-26 Houston Astros Game 2      2024-09-29 

In [61]:
# Function to check for type errors in columns
def check_type_errors(df):
    type_errors = {}
    for column in df.columns:
        try:
            df[column].astype(df[column].dtype)
        except ValueError as e:
            type_errors[column] = str(e)
    return type_errors

# Check for type errors in the data
type_errors = check_type_errors(data)
print("Columns with type errors:", type_errors)

Columns with type errors: {}


In [62]:
# Get the data types of all columns
column_types = data.dtypes
for column, dtype in column_types.items():
    print(f"{column}: {dtype}")

KeepFlag: int64
GlobalKey: int64
SeasonKey: int64
FanSinceDate: object
FirstGameAttended: object
TotalGamesAttended: float64
FirstGameBought: object
LastGameBought: object
TotalTicketsPurchased: float64
TotalLifetimeValue: float64
CurrentSeasonEmailActivities: float64
PreviousSeasonsEmailActivities: float64
STMFlagCurr: int64
TicketingFanType: object
EmailFanType: object
FullSeasonBuyer: float64
HalfSeasonBuyer: float64
QuarterSeasonBuyer: float64
MiniPlanBuyer: float64
IndividualGameBuyer: float64
City: object
State: object
PostalCd: object
Country: object
Gender: object
Education: object
Occupation: object
Age: float64
MaritalStatus: object
PresenceOfChildren: object
DwellingType: object
HouseholdIncome: object
NetWorth: object
PrimaryVehicleType: object
MSADescription: object
MailSuppresionFlg: float64
WorkingWomanFlg: float64
BankCardHolderFlg: float64
GasDepartmentRetailCardHolderFlg: float64
TravelEntertainmentCardHolderFlg: float64
CreditCardHolderUnknownTypeFlg: float64
Premium

<h3>Cleaning State Values</h3>
<p>Keep states, provinces, and territories in US and Canada</p>
<p>Other locations will be put in 'OTHERS' category

In [63]:
state_counts = data['State'].value_counts()
state_counts = state_counts.reset_index()
state_counts.columns = ['State', 'Count']
print(state_counts.to_string())

                            State   Count
0                              MI  198554
1                              OH   11709
2                              ON    7507
3                        MICHIGAN    6377
4                              FL    6121
5                              NY    4368
6                              CA    3742
7                              IL    3582
8                              IN    3166
9                              TX    2537
10                        FLORIDA    1762
11                           OHIO    1703
12                             PA    1691
13                       ILLINOIS    1310
14                             MA    1262
15                             NC    1234
16                             VA    1229
17                             AZ    1188
18                             GA    1173
19                     CALIFORNIA    1160
20                        ONTARIO    1154
21                             TN    1036
22                             NJ 

In [64]:
# Mapping dictionary for state normalization
state_mapping = {
    'MI': 'MICHIGAN',
    'OH': 'OHIO',
    'IL': 'ILLINOIS',
    'IN': 'INDIANA',
    'WI': 'WISCONSIN',
    'MN': 'MINNESOTA',
    'IA': 'IOWA',
    'KY': 'KENTUCKY',
    'MO': 'MISSOURI',
    'PA': 'PENNSYLVANIA',
    'NY': 'NEW YORK',
    'NJ': 'NEW JERSEY',
    'CT': 'CONNECTICUT',
    'MD': 'MARYLAND',
    'VA': 'VIRGINIA',
    'NC': 'NORTH CAROLINA',
    'SC': 'SOUTH CAROLINA',
    'GA': 'GEORGIA',
    'FL': 'FLORIDA',
    'TX': 'TEXAS',
    'CA': 'CALIFORNIA',
    'WA': 'WASHINGTON',
    'OR': 'OREGON',
    'CO': 'COLORADO',
    'AZ': 'ARIZONA',
    'NV': 'NEVADA',
    'UT': 'UTAH',
    'ID': 'IDAHO',
    'MT': 'MONTANA',
    'WY': 'WYOMING',
    'ND': 'NORTH DAKOTA',
    'SD': 'SOUTH DAKOTA',
    'NE': 'NEBRASKA',
    'KS': 'KANSAS',
    'LA': 'LOUISIANA',
    'AR': 'ARKANSAS',
    'MS': 'MISSISSIPPI',
    'AL': 'ALABAMA',
    'TN': 'TENNESSEE',
    'WV': 'WEST VIRGINIA',
    'DE': 'DELAWARE',
    'VT': 'VERMONT',
    'NH': 'NEW HAMPSHIRE',
    'ME': 'MAINE',
    'HI': 'HAWAII',
    'AK': 'ALASKA',
    'NM': 'NEW MEXICO',
    'OK': 'OKLAHOMA',
    'MA': 'MASSACHUSETTS',
    'RI': 'RHODE ISLAND',
    # Non-state territories and regions
    'PR': 'PUERTO RICO',
    'DC': 'DISTRICT OF COLUMBIA',
    'AS': 'AMERICAN SAMOA',
    'GU': 'GUAM',
    'MP': 'NORTHERN MARIANA ISLANDS',
    'VI': 'VIRGIN ISLANDS',
    'FM': 'FEDERATED STATES OF MICRONESIA',
    'MH': 'MARSHALL ISLANDS',
    # International regions (if applicable)
    'OT': 'ONTARIO',  # Example for Ontario, Canada
    'ON': 'ONTARIO',  # Example for Ontario, Canada
    'Ontario': 'ONTARIO',  # Example for Ontario, Canada
    'NS': 'NOVA SCOTIA',  # Example for Nova Scotia, Canada
    'QC': 'QUEBEC',  # Example for Quebec, Canada
    'BC': 'BRITISH COLUMBIA',  # Example for British Columbia, Canada
    'AB': 'ALBERTA',  # Example for Alberta, Canada
    'SK': 'SASKATCHEWAN',  # Example for Saskatchewan, Canada
    'NL': 'NEWFOUNDLAND AND LABRADOR',  # Example for Newfoundland and Labrador, Canada
    'YT': 'YUKON',  # Example for Yukon, Canada
    'NT': 'NORTHWEST TERRITORIES',  # Example for Northwest Territories, Canada
    'NU': 'NUNAVUT',  # Example for Nunavut, Canada
    'MB': 'MANITOBA',  # Example for Manitoba, Canada
    'PE': 'PRINCE EDWARD ISLAND',  # Example for Prince Edward Island, Canada
    'NB': 'NEW BRUNSWICK'  # Example for New Brunswick, Canada
}

# Function to normalize state values
def normalize_state(state):
    state = state.strip().upper()
    return state_mapping.get(state, state)

# Apply the normalization function to the 'State' column
data['State'] = data['State'].apply(lambda x: normalize_state(x) if pd.notnull(x) else x)

# Display the normalized state counts
normalized_state_counts = data['State'].value_counts().reset_index()
normalized_state_counts.columns = ['State', 'Count']
print(normalized_state_counts.to_string())

                            State   Count
0                        MICHIGAN  204931
1                            OHIO   13412
2                         ONTARIO    8661
3                         FLORIDA    7883
4                        NEW YORK    5013
5                      CALIFORNIA    4902
6                        ILLINOIS    4892
7                         INDIANA    3827
8                           TEXAS    3359
9                    PENNSYLVANIA    2152
10                       VIRGINIA    1778
11                 NORTH CAROLINA    1732
12                        ARIZONA    1656
13                        GEORGIA    1562
14                      TENNESSEE    1459
15                  MASSACHUSETTS    1430
16                     NEW JERSEY    1285
17                      WISCONSIN    1158
18                       COLORADO    1084
19                       MARYLAND     968
20                       MISSOURI     847
21                       KENTUCKY     841
22                 SOUTH CAROLINA 

In [65]:
# Get the counts of each state
state_counts = data['State'].value_counts()

# Function to update state values based on their occurrences
def update_state(state):
    if state not in state_mapping.values():
        return 'OTHERS'
    return state

# Function to group all values that are null as NULL
def update_Nullstate(state):
    if pd.isnull(state):
        return 'NULL'
    return state

# Apply the update function to the 'State' column
data['State'] = data['State'].apply(lambda x: update_state(x) if pd.notnull(x) else x)
data['State'] = data['State'].apply(lambda x: update_Nullstate(x))

# Display the updated state counts
updated_state_counts = data['State'].value_counts().reset_index()
updated_state_counts.columns = ['State', 'Count']
print(updated_state_counts.to_string())

                        State   Count
0                    MICHIGAN  204931
1                        NULL  110845
2                        OHIO   13412
3                     ONTARIO    8661
4                     FLORIDA    7883
5                    NEW YORK    5013
6                  CALIFORNIA    4902
7                    ILLINOIS    4892
8                     INDIANA    3827
9                       TEXAS    3359
10               PENNSYLVANIA    2152
11                   VIRGINIA    1778
12             NORTH CAROLINA    1732
13                    ARIZONA    1656
14                    GEORGIA    1562
15                  TENNESSEE    1459
16              MASSACHUSETTS    1430
17                     OTHERS    1333
18                 NEW JERSEY    1285
19                  WISCONSIN    1158
20                   COLORADO    1084
21                   MARYLAND     968
22                   MISSOURI     847
23                   KENTUCKY     841
24             SOUTH CAROLINA     821
25          

<h3>Redo sections that analysts did to keep global key present</h3>

<h5>Gender</h5>

In [66]:
#Will create three columns: isMale, isFemale, isNaN based on gender column
#Print values in Gender column
print(data['Gender'].value_counts())
#Create new columns
data['isMale'] = data['Gender'].apply(lambda x: 1 if x == 'Male' else 0)
data['isFemale'] = data['Gender'].apply(lambda x: 1 if x == 'Female' else 0)
#isNan should have all that are not 'Male' or 'Female' or is Null
data['isNaN'] = data['Gender'].apply(lambda x: 1 if pd.isnull(x) else 0)
# Find about 10,000 more non NaN and not Male or Female and lump them into isNaN
non_male_female = data[(data['Gender'] != 'Male') & (data['Gender'] != 'Female') & (pd.notnull(data['Gender']))]
data.loc[non_male_female.index, 'isNaN'] = 1

print(data[['isMale', 'isFemale', 'isNaN']].sum())

# confirm no overlap between isMale, isFemale, and isNaN
# Confirm no overlap between isMale, isFemale, and isNaN
assert data[['isMale', 'isFemale', 'isNaN']].sum().sum() == data.shape[0]

# Ensure the sum of all the columns values equals the number of rows
assert data[['isMale', 'isFemale', 'isNaN']].sum(axis=1).eq(1).all()

Gender
Male         158198
Female        95983
<Unknown>     77584
Name: count, dtype: int64
isMale      158198
isFemale     95983
isNaN       143006
dtype: int64


In [67]:
# Drop the gender column
data.drop(columns=['Gender'], inplace=True)

<h5>Age Redo</h5>

In [68]:
#print columns list
column_types = data.dtypes
for column, dtype in column_types.items():
    print(f"{column}: {dtype}")

KeepFlag: int64
GlobalKey: int64
SeasonKey: int64
FanSinceDate: object
FirstGameAttended: object
TotalGamesAttended: float64
FirstGameBought: object
LastGameBought: object
TotalTicketsPurchased: float64
TotalLifetimeValue: float64
CurrentSeasonEmailActivities: float64
PreviousSeasonsEmailActivities: float64
STMFlagCurr: int64
TicketingFanType: object
EmailFanType: object
FullSeasonBuyer: float64
HalfSeasonBuyer: float64
QuarterSeasonBuyer: float64
MiniPlanBuyer: float64
IndividualGameBuyer: float64
City: object
State: object
PostalCd: object
Country: object
Education: object
Occupation: object
Age: float64
MaritalStatus: object
PresenceOfChildren: object
DwellingType: object
HouseholdIncome: object
NetWorth: object
PrimaryVehicleType: object
MSADescription: object
MailSuppresionFlg: float64
WorkingWomanFlg: float64
BankCardHolderFlg: float64
GasDepartmentRetailCardHolderFlg: float64
TravelEntertainmentCardHolderFlg: float64
CreditCardHolderUnknownTypeFlg: float64
PremiumCardHolderFlg: 

In [69]:
# print values of data['Age']
print(data['Age'].value_counts())

Age
44.0    13220
42.0    13207
40.0    12605
46.0    12600
52.0    12082
48.0    11717
50.0    11458
38.0    11356
54.0    10690
56.0    10101
36.0    10059
60.0     9585
58.0     9536
62.0     9147
64.0     8670
34.0     8659
32.0     8409
66.0     7869
30.0     7618
28.0     7311
68.0     6698
26.0     6600
24.0     6030
70.0     5685
22.0     4948
72.0     4456
74.0     3570
20.0     3254
76.0     2628
78.0     1498
18.0     1476
80.0     1249
82.0      723
84.0      540
86.0      387
99.0      281
88.0      257
90.0      216
92.0      185
94.0      133
96.0      124
98.0       78
Name: count, dtype: int64


In [70]:
# print how many have null values
print(data['Age'].isnull().sum())

140272


In [71]:
import pandas as pd
from sklearn.cluster import KMeans

# Assuming 'data' is your DataFrame
# Use k-means clustering to fill in missing values for the 'Age' column

# Get the indices of rows with missing 'Age' values
missingAgeIndices = data[data['Age'].isnull()].index

# Get the indices of rows with non-missing 'Age' values
nonMissingAgeIndices = data[data['Age'].notnull()].index

# Create a k-means model with 3 clusters
kmeans = KMeans(n_clusters=3, random_state=0)

# Fit the model on the non-missing 'Age' values
kmeans.fit(data.loc[nonMissingAgeIndices, ['Age']])

# Predict the clusters for all 'Age' values
ageClusters = kmeans.predict(data[['Age']].fillna(0))

# Get the cluster centers
clusterCenters = kmeans.cluster_centers_

# Create a DataFrame to map clusters to their centers
clusterMap = pd.DataFrame(clusterCenters, columns=['Age'])

# Assign the cluster center values to the missing 'Age' values
for idx in missingAgeIndices:
    cluster = ageClusters[idx]
    data.at[idx, 'Age'] = clusterMap.at[cluster, 'Age']

# Now 'data' has the missing 'Age' values filled in using k-means clustering

/home/jbrinkm/IllitchML/venv/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [72]:
# print how many have null values
print(data['Age'].isnull().sum())

0


<h5>DONE!</h5>

<h5>Household Estimate Income</h5>

In [ ]:
# call binConvert function from Helpers.py
data = binConvert(data, 'HouseholdIncome')

In [135]:
import importlib
import Helpers
importlib.reload(Helpers)

<module 'Helpers' from '/home/jbrinkm/IllitchML/Helpers.py'>

In [132]:
# validate the conversion
print(getConvertedValues(data, 'HouseholdIncome'))

NameError: name 'getConvertedValues' is not defined

<h5>Net Worth</h5>

In [77]:
# call binConvert function from Helpers.py
data = binConvert(data, 'NetWorth')


In [130]:
# validate the conversion
print(getConvertedValues(data, 'HouseholdIncome'))

NameError: name 'getConvertedValues' is not defined

<h5>CurrentSeasonEmailActivities</h5>

In [134]:
print(data['CurrentSeasonEmailActivities'].unique())
print(data['CurrentSeasonEmailActivities'])

[ 10.  nan   1.   7.  13.   0.  25.   8.  21.  12.   2.   5.  19.  31.
  16.  28.  11.  17.  14.  24.   4.  27.  29.  64.  26.  20.  22.   9.
   6.  30.  32.  33.  34.  18.   3.  58. 129.  69.  15.  35.  49.  43.
  23.  54.  44.  41.  59.  39.  62.  57.  53.  60.  36.  38.  42.  45.
  37.  40.  52.  83.  63.  61.  71.  75. 100.  77.  56.  50.  51.  55.
  46.  66. 124.  48.  47.  65.  82.  76.  86.  79.  68.  67.  81.  72.
 156.  85.  74. 107.  84.  70. 143.  92.  99.  73. 106. 108. 105. 109.
  98. 171.  80.  90. 103. 147. 120.  93. 114.  88.  95.  87.  78.  94.
 133. 101. 111. 122. 123. 117. 102. 116.  96. 167.  89. 155. 175. 104.]
0         10.0
1          NaN
2          1.0
3          7.0
4         13.0
          ... 
397182     NaN
397183     NaN
397184     NaN
397185     NaN
397186     NaN
Name: CurrentSeasonEmailActivities, Length: 397187, dtype: float64


Doing nearest neighbor estimate for this

In [ ]:
nearestNeighborEstimate(data, 'CurrentSeasonEmailActivities')

<h1>Save work to TigersCleaned.csv</h1>

In [ ]:
#save work to a new csv file
data.to_csv('data/TigersCleaned.csv', index=False)

<h5>Cluster Analysis w/5 clusters</h5>

In [ ]:
# print all columns with NaN values
nan_columns = data.columns[data.isnull().any()]
# print the name of the columns with NaN values
for nanCol in nan_columns:
    print(nanCol)

FirstGameAttended
TotalGamesAttended
FirstGameBought
LastGameBought
TotalTicketsPurchased
TotalLifetimeValue
CurrentSeasonEmailActivities
PreviousSeasonsEmailActivities
FullSeasonBuyer
HalfSeasonBuyer
QuarterSeasonBuyer
MiniPlanBuyer
IndividualGameBuyer
City
PostalCd
Country
Education
Occupation
MaritalStatus
PresenceOfChildren
DwellingType
NetWorth
PrimaryVehicleType
MSADescription
MailSuppresionFlg
WorkingWomanFlg
BankCardHolderFlg
GasDepartmentRetailCardHolderFlg
TravelEntertainmentCardHolderFlg
CreditCardHolderUnknownTypeFlg
PremiumCardHolderFlg
UpscaleDepartmentStoreCardHolderFlg
MailOrderResponderFlg
TruckOwnerFlg
MotorcycleOwnerFlg
RVOwnerFlg
IntTheatrePerformingArtsFlg
IntArtsFlg
IntDomesticTravelFlg
IntHomeStereosFlg
IntMusicDevicesFlg
IntMusicAvidListenerFlg
IntMusicCollectorFlg
IntMovieCollectorFlg
IntAutoRacingFlg
IntFootballFlg
IntBaseballFlg
IntBasketballFlg
IntHockeyFlg
IntContestsFlg
IntSportsFlg
IntMusicMoviesFlg
IntNascarFlg
IntUpscaleLivingFlg
DistanceToArena
NumberO

In [ ]:
# drop nan (will eventually handle with cleaning)
data_clean = data.dropna()
print(data_clean.shape)

(0, 217)


In [ ]:
import pandas as pd
from sklearn.mixture import GaussianMixture

# Assuming 'data' is your DataFrame

# Create a Gaussian Mixture Model with 5 components (clusters)
gmm = GaussianMixture(n_components=5, random_state=0)

# Fit the model on the entire dataset (excluding any non-numeric columns)
gmm.fit(data_clean.select_dtypes(include=[float, int]))

# Predict the clusters for all data points
data_clean['Cluster'] = gmm.predict(data_clean.select_dtypes(include=[float, int]))

# Get the cluster centers (means of the Gaussian components)
clusterCenters = gmm.means_

# Create a DataFrame to map clusters to their centers
clusterMap = pd.DataFrame(clusterCenters, columns=data_clean.select_dtypes(include=[float, int]).columns)

# Now 'data' has an additional 'Cluster' column indicating the cluster each data point belongs to

ValueError: Found array with 0 sample(s) (shape=(0, 190)) while a minimum of 2 is required by GaussianMixture.